# Fundamental Properties of EMU Radio Sources - Cross-Match Catalogs using NWAY

## Libraries

In [ ]:
from astropy.io import fits
import numpy as np
import subprocess

## Import Catalog Data

In [ ]:
emu_catalog_path = "/home/polaris/Lab_Astro/data/G23-ASKAP-EMUES-master-cat.fits"
emu_mosaic_path = "/home/polaris/Lab_Astro/data/G23-ASKAP-EMUES-data/G23-EMUES-mosaic.fits"

gama_catalog_path = "/home/polaris/Lab_Astro/data/gkvInputCatv02.fits"

In [ ]:
def get_catalog_data(catalog_path):
    # Open the fits file of the catalog as a fits object
    catalog = fits.open(catalog_path)

    # Get the data and header of the catalog
    catalog_data = catalog[1].data
    catalog_header = catalog[1].header
    
    return catalog_data, catalog_header


In [ ]:
emu_catalog_data, emu_catalog_header = get_catalog_data(emu_catalog_path)
gama_catalog_data, gama_catalog_header = get_catalog_data(gama_catalog_path)

## Prepare the catalogs for NWAY input
The NWAY library has multiple requirements for a catalog to serve as input.
1. The fits data table needs to have a extension name, that is used as a prefix for the columns copied to the output catalog
2. The fits header of the catalogs needs to have the SKYAREA keyword. This keyword has the area on the sky in square degrees covered by the catalog. Since I don't have the documentation for EMU data, I must calculate the area using the mosaic image.
3. The position error needs to be a column with a single value, called positional error, so we need to obtain it from the right ascension and declination errors.

### Change extension name
This process is also possible using just the NWAY library. Check page 8 of the NWAY documentation

In [ ]:
def change_hdu_name(fits_file, hdu_index, new_name):
    with fits.open(fits_file, mode='update') as hdul:
        # Change the name of the specified HDU
        hdul[hdu_index].header['EXTNAME'] = new_name
        # Save changes to the FITS file
        hdul.flush()

Change extension name of EMU survey to "EMU" and of GAMA-23 survey to "G23"

In [ ]:
change_hdu_name(emu_catalog_path, hdu_index=1, new_name="EMU")
change_hdu_name(gama_catalog_path, hdu_index=1, new_name="G23")

### Calculate the area of the survey
Given a fits image of a survey, calculate the area of the survey in degrees^2. The fits image is in pixels and the CDELT1 value on the header is used to rescale to physical units.

In [ ]:
def calculate_survey_area(fits_file):
    # Open the FITS file and extract the header
    with fits.open(fits_file) as hdul:
        header = hdul[0].header
        data = hdul[0].data
        
        # Ensure data is at least 2D (handle multi-dimensional FITS files)
        if data.ndim > 2:
            data = data[0]  # Take the first frame if it's a multi-frame FITS
        
        # Get image dimensions
        height, width = data.shape[-2:]  # Ensure correct shape indexing
        total_pixels = width * height
        
        # Extract pixel scale from header (if available)
        if 'PIXSCALE' in header:
            pixel_scale = header['PIXSCALE']  # in arcsec/pixel
        elif 'CDELT1' in header and 'CDELT2' in header:
            if header['CUNIT1'] == 'deg' and header['CUNIT2'] == 'deg': # Check if units are in degrees
                pixel_scale = np.abs(header['CDELT1']) * 3600  # Convert from decimal degrees to arcseconds
        else:
            raise ValueError("Pixel scale not found in FITS header. Check metadata or specify manually.")
        
        # Compute the survey area in square arcseconds
        area_arcsec2 = total_pixels * (pixel_scale ** 2)
        
        # Convert to square degrees
        area_deg2 = area_arcsec2 / (3600 ** 2)
        
    print(f"Total pixels: {total_pixels}")
    print(f"Pixel scale (arcsec/pixel): {pixel_scale}")
    print(f"Survey area (deg^2): {area_deg2}")


In [ ]:
area_emu_survey = calculate_survey_area(emu_mosaic_path)
area_emu_survey

### Create positional error column and append to fits file
This function considers that both the error in RA and DEC are in angular units. The positional error is calculated by using the uncertainty formula.

In [ ]:
def create_positional_error_column(fits_file):
    # Open the FITS file and extract data
    with fits.open(fits_file, mode='update') as hdul:
        data = hdul[1].data

        ra_error = data['E_RA']
        dec_error = data['E_DEC']

        # Compute the positional error
        pos_error_data = np.sqrt(ra_error ** 2 + dec_error ** 2)

        # Create a new column
        pos_error_col = fits.Column(name='POS_ERROR', format='D', array=pos_error_data) # 'D' is for double precision float (64-bit)

        # Add the new column to the existing table
        new_columns = hdul[1].columns + fits.ColDefs([pos_error_col])
        new_hdu = fits.BinTableHDU.from_columns(new_columns)

        # Replace the binary table HDU with the updated one
        hdul[1] = new_hdu

        # Save changes to the FITS file
        hdul.flush()

        print("Positional error column created successfully.")

In [ ]:
create_positional_error_column(emu_catalog_path)

## Run NWAY

In [ ]:
def run_nway_and_print_output(primary_catalog, secondary_catalog, output_file, primary_err_col, secondary_err_col):
    """
    Run NWAY for cross-matching catalogs and print the output.

    Parameters:
    primary_catalog (str): Path to the primary catalog FITS file.
    secondary_catalog (str): Path to the secondary catalog FITS file.
    output_file (str): Path to save the output catalog.
    primary_err_col (str): Column name for positional error in the primary catalog.
    secondary_err_col (str): Column name for positional error in the secondary catalog.
    """
    # Construct the NWAY command
    command = [
        'nway.py ',
        primary_catalog, 
        ' :', primary_err_col, ' ',
        secondary_catalog,
        ' :', secondary_err_col,
        " --out", output_file,
    ]

    # Run the NWAY command and capture output
    try:
        result = subprocess.run(command, check=True, text=True, capture_output=True)
        print("NWAY Output:")
        print(result.stdout)  # Print the standard output from NWAY
        if result.stderr:
            print("NWAY Errors:")
            print(result.stderr)  # Print any errors from NWAY
    except subprocess.CalledProcessError as e:
        print(f"Error running NWAY: {e}")
        print(f"Standard Output:\n{e.stdout}")
        print(f"Standard Error:\n{e.stderr}")
    except FileNotFoundError:
        print("NWAY is not installed or not found in the system PATH.")
